<a href="https://colab.research.google.com/github/JosueAfouda/MLflow/blob/main/first_model_with_MLflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#pip install mlflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 17.9 MB 471 kB/s 
     |████████████████████████████████| 146 kB 56.3 MB/s 
     |████████████████████████████████| 209 kB 53.6 MB/s 
     |████████████████████████████████| 596 kB 50.8 MB/s 
     |████████████████████████████████| 81 kB 5.6 MB/s 
     |████████████████████████████████| 181 kB 37.8 MB/s 
     |████████████████████████████████| 79 kB 6.6 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 78 kB 7.1 MB/s 
  Created wheel for databricks-cli: filename=databricks_cli-0.17.0-py3-none-any.whl size=141960 sha256=6c31a7f91ea6d8db1dd0c9859e54c968a9a99f0e9ecdd87dc7f02e364de8b677
  Stored in directory: /root/.cache/pip/wheels/55/c3/db/33705569425fd2bdc9ea73051a8053fa26965c2bce8a146747
Successfully built databricks-cli
  Attempting uninstall

In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
dataset = datasets.load_iris()
seed = 123
X_train, X_test, y_train, y_test = train_test_split(
    dataset.data,
    dataset.target,
    test_size = 0.4,
    random_state = seed
)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(90, 4)
(90,)
(60, 4)
(60,)


In [2]:
X_train.shape

(90, 4)

In [3]:
dataset.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [4]:
y_train

array([0, 2, 1, 0, 0, 2, 1, 2, 2, 0, 1, 1, 2, 0, 2, 1, 1, 0, 2, 2, 0, 0,
       1, 1, 2, 0, 0, 1, 0, 1, 2, 0, 2, 0, 0, 1, 0, 0, 1, 2, 1, 1, 1, 0,
       0, 1, 2, 0, 0, 1, 1, 1, 2, 1, 1, 1, 2, 0, 0, 1, 2, 2, 2, 2, 0, 1,
       0, 1, 1, 0, 1, 2, 1, 2, 2, 0, 1, 0, 2, 2, 1, 1, 2, 2, 1, 0, 1, 1,
       2, 2])

In [5]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_train, y_train)

C:\Users\Utilisateur\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

En ajoutant quelques lignes de code au script précédent, on peut déjà enregistrer une première expérience avec MLflow.

In [6]:
from sklearn.linear_model import LogisticRegression
import mlflow
# Définition d'une expérience (Une expérience peut contenir plusieurs *runs*)
mlflow.set_experiment("Reg_Logistic_Models")
mlflow.sklearn.autolog() # Ici on fait appel à l'API MLflow qui intégre les algos de Scikit-Learn.
                        # C'est grâce à ce code qu'on enregistre toutes les métriques et métadonnées 
                      #de cette expérience (Module MLflow Tracking)

# Création du premier  run dans la même expérience
with mlflow.start_run(run_name = 'reg_logistic_defaut'):
    clf = LogisticRegression()
    clf.fit(X_train, y_train)

C:\Users\Utilisateur\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [7]:
# Création d'un second run dans la même expérience
with mlflow.start_run(run_name = 'reg_logistic_max_iter_200'):
    clf2 = LogisticRegression(max_iter=200)
    clf2.fit(X_train, y_train)

In [7]:
# Création d'une seconde expérience
mlflow.set_experiment("Decision_Tree_Models")
mlflow.sklearn.autolog()
from sklearn.tree import DecisionTreeClassifier 
with mlflow.start_run(run_name = 'decision_tree_defaut'):
    dt1 = DecisionTreeClassifier(random_state=seed)
    dt1.fit(X_train, y_train)

In [8]:
with mlflow.start_run(run_name = 'decision_tree_max_depth3'):
    dt2 = DecisionTreeClassifier(random_state=seed, max_depth=3)
    dt2.fit(X_train, y_train)

In [9]:
import numpy as np
new_data = np.array([[6.5, 3.2, 5.5, 1.9]])
new_data

array([[6.5, 3.2, 5.5, 1.9]])

In [10]:
new_data.shape

(1, 4)

In [16]:
import mlflow
logged_model = 'runs:/d0e445718203479da15230b64ba8bf07/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
prediction= loaded_model.predict(pd.DataFrame(new_data))
prediction

array([2])

In [14]:
# Prédictions
logged_model = 'runs:/12366ea03957424891b6f90e65086664/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
prediction = loaded_model.predict(new_data)
prediction

MlflowException: Run '12366ea03957424891b6f90e65086664' not found

In [24]:
# Prédictions
logged_model = 'runs:/12366ea03957424891b6f90e65086664/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
prediction = loaded_model.predict(new_data)
prediction

array([2])

In [9]:
#pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 7.7 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=b95ed2f31c52a6a3268ddb2fc515c356dd4f39da6423a121e44ed4f8718d93e0
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [12]:
from pyngrok import ngrok
# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2ZiHJoOfGjrsTKd5PxLfH9HevEj_3k2yBAT1nkhtsUvMPdhE5" 
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

MLflow Tracking UI: https://d6b0-105-235-71-131.ngrok-free.app                                      


t=2023-12-18T11:17:53+0000 lvl=warn msg="failed to open private leg" id=55acaf9b5089 privaddr=localhost:5000 err="dial tcp [::1]:5000: connectex: No connection could be made because the target machine actively refused it."
t=2023-12-18T11:17:54+0000 lvl=warn msg="failed to open private leg" id=fde2df63f2bc privaddr=localhost:5000 err="dial tcp [::1]:5000: connectex: No connection could be made because the target machine actively refused it."


In [ ]:
!mlflow ui

t=2023-12-18T12:29:45+0000 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=ecb2a9ddb0d1 err="read tcp 172.16.2.172:51682->3.134.73.173:443: wsarecv: An established connection was aborted by the software in your host machine."
t=2023-12-18T12:29:45+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session obj=csess id=ecb2a9ddb0d1 err="failed to dial ngrok server with address \"connect.us.ngrok-agent.com:443\": dial tcp: lookup connect.us.ngrok-agent.com: no such host"
t=2023-12-18T12:29:46+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session obj=csess id=ecb2a9ddb0d1 err="failed to dial ngrok server with address \"connect.us.ngrok-agent.com:443\": dial tcp: lookup connect.us.ngrok-agent.com: no such host"
t=2023-12-18T12:29:47+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session obj=csess id=ecb2a9ddb0d1 err="failed to dial ngrok server with address \"connect.us.ngrok-agent.com:443\": dial tcp: lookup conne